In [1]:
from mesh import OrganMesh
from scipy.spatial.transform import Rotation
import numpy as np
from copy import deepcopy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [19]:
tetra = OrganMesh("/work/3D-CT PLY/liver.ply")
tetra.save('tmp.ply')
loaded_mesh = OrganMesh('tmp.ply')
for i in range(tetra.num_nodes):
    np.testing.assert_array_almost_equal(tetra.nodes[i], loaded_mesh.nodes[i], decimal=6)

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 1 / 3 (33.3%)
Max absolute difference: 3.78417968e-06
Max relative difference: 3.43214449e-08
 x: array([-110.257 ,   26.3379,  -39.2795])
 y: array([-110.257004,   26.3379  ,  -39.279499])

In [3]:
# def translate(target):
#     tmp = deepcopy(target)
#     rotvec = np.array([0, np.pi / 2, np.pi / 3])
#     rot = Rotation.from_rotvec(rotvec)
#     rotation_matrix = rot.as_matrix()
#     translate = np.array([1,1,1])
#     for i in range(target.num_nodes):
#         n = target.nodes[i]
#         tmp.nodes[i] =  rotation_matrix @ n + translate
#     return tmp
    
# test_liver = OrganMesh("/work/3D-CT PLY/liver.ply")
# translated_liver = translate(test_liver)
# translated_liver.save("/work/test/test_translate_liver.ply")
# tetra.save("/work/test/test_tetra.ply")

In [2]:
import sys
sys.path.append('/work/src')
from registration import GlobalRotationRegisration, PointToPointICP, PointToPointTrimeshICP

In [3]:
source = OrganMesh("/work/3D-CT PLY/liver.ply")
target = OrganMesh("/work/test/test_translate_liver.ply")
registration = PointToPointICP(source=source, target=target)
p2p_transformed_mesh = registration.transform()

def evaluate(source, target):
    err = 0.
    for i in range(source.num_nodes):
        err += np.linalg.norm(source.nodes[i] - target.nodes[i])
    return err / source.num_nodes

In [4]:
evaluate(p2p_transformed_mesh, target)

177.36377353291053

In [5]:
global_rotation_registration = GlobalRotationRegisration(source=source, target=target)
gr_transformed_mesh = global_rotation_registration.transform()
evaluate(gr_transformed_mesh, target)
gr_transformed_mesh.save('/work/test/gr_transformed_liver.ply')

[8.71998877e-10 1.57079633e+00 1.04719755e+00]


In [6]:

evaluate(gr_transformed_mesh, target)

2.4216268084890235e-06

In [17]:
p2p_trimesh_regisration = PointToPointTrimeshICP(source=source, target=target)
p2p_trimesh_regisrationed = p2p_trimesh_regisration.transform()
print(evaluate(p2p_trimesh_regisrationed, target))
p2p_trimesh_regisrationed.save('/work/test/trimeshP2P_transformed_liver.ply')

106.60199441075419


In [18]:
evaluate(source, target)

151.02248265125777

In [19]:
evaluate(source, source)

0.0

In [20]:
!pip install rtree

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [21]:

from registration import TrimeshPCA


trimesh_pca = TrimeshPCA(source=source, target=target)
trimesh_pca_transformed = trimesh_pca.transform()
print( evaluate(trimesh_pca_transformed, target))
trimesh_pca_transformed.save('/work/test/gr_transformed_liver.ply')

64.75069035381225


# 問題点
- Meshの位置合わせができない。
 - 頂点の対応関係は？
  - おそらく取れているはず。確証はない。
  

In [22]:
tetra

NameError: name 'tetra' is not defined